In [1]:
## Import packages
import optuna
import warnings
import numpy as np
import pandas as pd
import seaborn as sns 
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from typing import Literal, Callable
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold

# To remove warnings
warnings.filterwarnings("ignore")

c:\Users\Siah Jin Thau\OneDrive\Desktop\Work\IT1244\Group Project\IT1244-Fraud-Analysis\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Data Importing functions

def import_client(filepath : str = "Data/client.csv") -> pd.DataFrame:
    """
    Function to import client dataset.
    """
    return pd.read_csv(filepath)

def import_invoice(filepath : str = "Data/invoice.csv") -> pd.DataFrame:
    """
    Function to import invoice dataset.
    """
    return pd.read_csv(filepath)

In [3]:
## Data Preprocessing functions

def convert_date(df: pd.DataFrame) -> pd.DataFrame:
    """
    Converts date column to an integer representing 
    the number of days since the earliest date.
    *Column name 'date' is fixed for both datasets.
    """

    # Convert 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'], dayfirst = True)
    
    # Find the earliest date
    earliest_date = df['date'].min()
    
    # Calculate the number of days since the earliest date
    df['date'] = (df['date'] - earliest_date).dt.days
    
    return df

def drop_duplicates(df : pd.DataFrame, logs : bool = False) -> pd.DataFrame:
    """
    Prints the result of a duplicate check.
    Drops duplicates if they exist.
    """
    if df.duplicated().any(): # Duplicates check
        if logs:
            print("Duplicates found! Cleaning them up...")
        df = df.drop_duplicates() # Drops duplicates
        df = df.reset_index(drop = True) # Resets indexes
    elif logs:
        print("No duplicates found!")
    return df

def convert_to_categorical(
        df : pd.DataFrame,
        cols : list[str]
        ) -> pd.DataFrame:
    """
    Converts list of column names to categorical datatype.
    """
    df[cols] = df[cols].astype('category')
    return df


In [4]:
## Data Feature Engineering functions

def aggregate_invoice(df : pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate the invoice dataframe by id and generate features.
    Calculates sum, mean, max, and std for each consommation_level and date
    and counts number of invoices under each id.
    """
    df = df.groupby('id').agg({ # Aggregate by id
        # Calculate sum, mean, max, and std for each consm_level and date
        'consommation_level_1': ['sum', 'mean', 'max', 'std'],
        'consommation_level_2': ['sum', 'mean', 'max', 'std'],
        'consommation_level_3': ['sum', 'mean', 'max', 'std'],
        'consommation_level_4': ['sum', 'mean', 'max', 'std'],
        'date': ['sum', 'mean', 'max', 'std'],
        'counter_statue': 'count', # Count number of invoices
    }).reset_index()
    df.columns = [
        # Join by _ if more than 2 parts to the column name exists
        '_'.join(col).strip() if col[1] 
        else col[0] # Else use original name
        for col in df.columns.values # For each column name value
    ]
    return df

def manual_fix_names(
        df : pd.DataFrame,
        new_col_names : list[str]
        ) -> pd.DataFrame:
    """
    Manually sets the column names of a dataframe.
    """
    df.columns = new_col_names
    return df


In [5]:
## Data Joining functions

def merge(
        client_df : pd.DataFrame,
        invoice_df : pd.DataFrame,
        merge_by : str = "id"
        ) -> pd.DataFrame:
    """
    Merges two dataframes.
    Merges on 'id' column by default (for client and invoice).
    """
    merged_df = pd.merge(
        client_df, invoice_df, on = merge_by)
    return merged_df


In [6]:
## Dimensionality Reduction functions

def prep_dataframe(
        df : pd.DataFrame, 
        response_col_name : str,
        cat_col_names : list[str] = [],
        scale : bool = True,
        OHE : bool = True
        ) -> pd.DataFrame:
    """
    For scaling and one-hot encoding dataframe.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        response_col_name (str): Name of response column
        cat_col_names (list[str]): List of categorical column names to evaluate.
        scale (bool): Indicate if data should be scaled
        OHE (bool): Indicate if categorical variables should undergo One-Hot Encoding

    Returns:
        pd.DataFrame: A DataFrame with columns scaled/One-Hot Encoded
    """

    df = df.drop(columns = ['id']) 
    # Drop id column as it is not useful for predicting

    cat_col_names.remove('id') 
    # Remove 'id' as it no longer exists in dataframe

    if response_col_name in cat_col_names:
        cat_col_names.remove(response_col_name)
    # Exclude doing OHE on response column

    # Identify numerical columns 
    # by excluding categorical and response columns
    num_col_names = [
        col for col in df.columns 
        if col not in cat_col_names 
        and col != response_col_name
    ]

    y = df[response_col_name].values # Response column
    X_num = df[num_col_names].values # Numerical columns
    X_cat = df[cat_col_names].values # Categorical columns

    if scale:
        # Scale numerical features
        scaler = StandardScaler()
        X_num = scaler.fit_transform(X_num)

    if OHE:
        # One-hot encode categorical features
        encoder = OneHotEncoder(sparse_output = False, drop = 'first')  
        # drop = 'first' to avoid dummy variable trap
        X_cat = encoder.fit_transform(X_cat)
    
    # Combine the scaled numerical and encoded categorical features
    X_prep = pd.DataFrame(
        data = np.hstack((X_num, X_cat)),  # Horizontal stack to combine arrays
        columns = num_col_names + list(encoder.get_feature_names_out(cat_col_names))
    )

    # Create a new DataFrame including the response variable
    prep_df = pd.DataFrame(X_prep, columns = X_prep.columns)
    prep_df[response_col_name] = y

    return prep_df

def principal_component_analysis(
        df : pd.DataFrame, 
        response_col_name : str,
        cat_col_names : list[str] = [], 
        var : float = 0.95, 
        logs : bool = False
        ) -> pd.DataFrame:
    """
    For reducing the dimensions of a dataframe using PCA.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        response_col_name (str): Name of response column
        cat_col_names (list[str]): List of categorical column names to evaluate.
        var (float): Proportion of variance that should be preserved
        logs (bool): Indicate if logs should be printed

    Returns:
        pd.DataFrame: A DataFrame that has undergone PCA
    """

    if response_col_name in cat_col_names:
        cat_col_names.remove(response_col_name)
    # Exclude doing OHE on response column

    # Identify numerical columns 
    # by excluding categorical and response columns
    num_col_names = [
        col for col in df.columns 
        if col not in cat_col_names 
        and col != response_col_name
    ]

    y = df[response_col_name].values # Response column
    X_num = df[num_col_names].values # Numerical columns
    X_cat = df[cat_col_names].values # Categorical columns

    pca = PCA(n_components = var) 
    # Keep 'var' proportion of the variance : default 95%
    X_pca = pca.fit_transform(X_num) 

    pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
    df_pca = pd.DataFrame(X_pca, columns = pca_columns)

    df_modified = pd.concat(
        [
            df_pca, 
            pd.DataFrame(X_cat, columns = cat_col_names), 
            pd.Series(y, name = response_col_name)
        ], 
        axis = 1)
    
    if logs:
        print(f"Number of components selected: {pca.n_components_}")
        print("Explained variance ratio for each component:", pca.explained_variance_ratio_)
        print("Cumulative explained variance:", pca.explained_variance_ratio_.cumsum())
        print("Final DataFrame with PCA applied to numeric columns:")
        print(df_modified.head())

    return df_modified

def filter_low_variance(
        df : pd.DataFrame, 
        response_col_name : str,
        threshold: float = 0.1
        ) -> pd.DataFrame:
    """
    Filter numeric columns based on a variance threshold.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        response_col_name (str): Name of response column
        threshold (float): The variance threshold for filtering columns.

    Returns:
        pd.DataFrame: A DataFrame with low variance numeric columns removed.
    """
    
    # Identify numerical columns 
    # by excluding response column
    num_col_names = [
        col for col in df.columns 
        if col != response_col_name
    ]

    # Calculate the variance for each numeric column
    variances = df[num_col_names].var()

    # Keep response column
    high_variance_cols = [response_col_name]

    # Filter columns with variance greater than the threshold : default 0.1
    cols_to_keep = variances[variances > threshold].index.tolist()
    high_variance_cols.extend(cols_to_keep)

    # Create a new DataFrame with only the selected columns
    filtered_df = df[high_variance_cols]

    return filtered_df


In [7]:
## Data Visualisation functions

def heatmap(
        df : pd.DataFrame,
        excluded_cols : list[str],
        annotate : bool = False
        ) -> None:
    """
    Plots a Correlation Heatmap.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        excluded_cols (list[str]): Columns to be excluded from the heatmap.
        annotate (float): To annotate the heatmap with correlation values.
    """
    
    # Identify desired columns 
    # by excluding undesired column
    col_names = [
        col for col in df.columns 
        if col not in excluded_cols
    ]

    X = df[col_names]

    corr_matrix = X.corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        corr_matrix, annot = annotate, cmap = 'coolwarm', fmt = '.2f'
    )
    plt.title('Correlation Heatmap')
    plt.show()


In [8]:
## Data Balancing functions

def undersample_data(
        df: pd.DataFrame, 
        response_col_name: str, 
        proportion: float = 1
        ) -> pd.DataFrame:
    """
    Perform undersampling to balance the classes in the dataset based on a specified proportion.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        response_col_name (str): The name of the target column (class label).
        proportion (float): The desired ratio of minority to majority class samples (e.g., 0.5 for 1:2).

    Returns:
        pd.DataFrame: A balanced DataFrame with samples from both classes according to the specified proportion.
    """

    # Separate the majority and minority classes
    majority_class = df[response_col_name].value_counts().idxmax()
    minority_class = df[response_col_name].value_counts().idxmin()

    # Split the DataFrame into majority and minority
    df_majority = df[df[response_col_name] == majority_class]
    df_minority = df[df[response_col_name] == minority_class]

    # Calculate the number of samples to take from the majority class
    n_minority = len(df_minority)
    n_majority = int(n_minority / proportion)

    # Ensure we do not sample more than available
    n_majority = min(n_majority, len(df_majority))

    # Randomly sample from the majority class
    df_majority_sampled = df_majority.sample(n = n_majority, random_state = 42)

    # Combine the minority class with the undersampled majority class
    df_balanced = pd.concat([df_majority_sampled, df_minority])

    # Shuffle the resulting DataFrame
    df_balanced = df_balanced.sample(frac = 1, random_state = 42).reset_index(drop = True)

    return df_balanced

def smote(df : pd.DataFrame, response_col_name : str) -> pd.DataFrame:
    """
    Apply SMOTE to oversample the minority class in the dataset and return a balanced DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        response_col_name (str): The name of the target column (class label).
        
    Returns:
        pd.DataFrame: A DataFrame with balanced data using SMOTE.
    """
    y = df[response_col_name]
    X = df.drop(columns=[response_col_name])
    
    sm = SMOTE(random_state = 42)
    X_resampled, y_resampled = sm.fit_resample(X, y)
    
    df_resampled = pd.concat(
        [
            pd.DataFrame(X_resampled, columns = X.columns), 
            pd.DataFrame(y_resampled, columns = [response_col_name])
        ], 
        axis = 1)
    
    return df_resampled

def balance_data(
        df : pd.DataFrame, 
        response_col_name : str,
        prop_synthetic_data : float = 0.25
        ) -> pd.DataFrame:
    """
    Apply SMOTE and undersampling to return a balanced DataFrame.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        response_col_name (str): The name of the target column (class label).
        prop_synthetic_data (float): Proportion of final data that will be synthetic. (0 to 0.5)

    Returns:
        pd.DataFrame: A DataFrame with balanced data.
    """

    if prop_synthetic_data < 0:
        prop_synthetic_data = 0
    elif prop_synthetic_data >= 0.5:
        prop_synthetic_data = 0.499

    df = undersample_data(
        df = df, 
        response_col_name = response_col_name,
        proportion = (1 - 2 * prop_synthetic_data))
    
    df = smote(df, response_col_name = response_col_name)

    return df

In [9]:
## Feature Importance function

def get_feature_importance(
        df : pd.DataFrame, 
        response_col_name : str,
        excluded_cols : list[str] = []
        ) -> None:
    """
    Uses Random Forests to identify the most important features in predicting response.

    Parameters:
        df (pd.DataFrame): The input DataFrame containing the data.
        response_col_name (str): The name of the target column (class label).
        excluded_cols (list[str]): Columns to be excluded from the heatmap.
        scale (bool): To scale the dataset or not.
    """

    col_names = [
        col for col in df.columns 
        if col not in excluded_cols 
        and col != response_col_name
    ]

    X = df[col_names]
    y = df[response_col_name]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size = 0.3, random_state = 42, stratify = y)
    
    # Initialised the Random Forest Classifier with class weight to handle class imbalance
    rf_model = RandomForestClassifier(class_weight = 'balanced', random_state = 42)

    # Train the model
    rf_model.fit(X_train, y_train)

    # Feature importance
    feature_importances = rf_model.feature_importances_

    # Create a DataFrame for feature importance
    features = X.columns  # Get feature names
    importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

    # Sort features by importance
    importance_df = importance_df.sort_values(by = 'Importance', ascending = False)

    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'], color='b')
    plt.gca().invert_yaxis()  # Highest importance at the top
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance in Random Forest')
    plt.show()


In [10]:
## Define Data Processing workflow

def data_processing(logs : bool = False) -> pd.DataFrame: 
    client_df = import_client()
    invoice_df = import_invoice()
    client_df = convert_date(client_df) # Convert date cols
    invoice_df = convert_date(invoice_df)
    # Drop duplicates rows
    client_df = drop_duplicates(client_df, logs = logs) 
    invoice_df = drop_duplicates(invoice_df, logs = logs)
    categorical_column_names = ['region', 'dis', 'id', 'catg', 'target']
    client_df = convert_to_categorical( # Convert categorical cols
        client_df, cols = categorical_column_names
        )
    invoice_df = aggregate_invoice(invoice_df) # Aggregate invoices
    invoice_df = manual_fix_names( # Fix column names manually
        invoice_df, 
        new_col_names = [
            'id', 
            'cons_level_1_sum', 'cons_level_1_mean', 
            'cons_level_1_max', 'cons_level_1_std',
            'cons_level_2_sum', 'cons_level_2_mean', 
            'cons_level_2_max', 'cons_level_2_std',
            'cons_level_3_sum', 'cons_level_3_mean', 
            'cons_level_3_max', 'cons_level_3_std',
            'cons_level_4_sum', 'cons_level_4_mean', 
            'cons_level_4_max', 'cons_level_4_std',
            'date_sum', 'date_mean', 'date_max', 'date_std',
            'num_invoices'
            ]
        )
    df = merge(client_df = client_df, invoice_df = invoice_df) # Merge
    df = prep_dataframe( # Prep for PCA
        df = df,
        response_col_name = 'target',
        cat_col_names = categorical_column_names
    )
    if logs:
        heatmap( # Do Heatmap for pre-reduced data
            df, excluded_cols = categorical_column_names)
        # Get feature importance for pre-reduced data
        get_feature_importance(df, 'target', categorical_column_names)
    df = principal_component_analysis( # Do PCA
        df = df,
        response_col_name = 'target'
    )
    # Do Low Variance Filter
    df = filter_low_variance(df, response_col_name = 'target')
    if logs:
        heatmap( # Do Heatmap for post-reduced data
            df, excluded_cols = categorical_column_names, annotate = True)
        # Get feature importance for post-reduced data
        get_feature_importance(df, 'target')
        # Print pre-balancing proportion of response
        print(df['target'].value_counts()) 
    df = balance_data( # Do balancing 
        df = df, 
        response_col_name = 'target', 
        prop_synthetic_data = 0.4 # Final proportion of synthetic data
        ) 
    if logs:
        # Print post-balancing proportion of response
        print(df['target'].value_counts()) 
        print(df.head())
    return df


In [11]:
## Define Models

def logistic_regression(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses logistic regression algorithm to do classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        
    Returns:
        np.ndarray: An array with model predictions.
    """

    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = LogisticRegression(random_state = 42)
    Model.fit(X = X, y = y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:, 1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred

def k_nearest_neighbors(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        k : int = 5,
        weights : Literal['uniform', 'distance'] = 'uniform',
        power_parameter : int = 2
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses k-nearest neighbors algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        k (int): Number of nearest neighbors to consider.
        weights ({'uniform', 'distance'}): Weight function used in prediction.
        power_parameter (int): Power parameter for the Minkowski metric. 
    
    Returns:
        np.ndarray: An array with model predictions.
    """
    
    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = KNeighborsClassifier(
        n_neighbors = k,
        weights = weights,
        p = power_parameter
    )
    Model.fit(X = X, y = y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:, 1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred

def decision_tree(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        criterion : Literal['gini', 'entropy', 'log_loss'] = 'gini',
        splitter : Literal['best', 'random'] = 'best',
        max_depth : int = None,
        min_samples_split : float = None,
        min_samples_leaf : float = None
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses decision tree algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        criterion ({'gini', 'entropy', 'log_loss'}): The function to measure the quality of a split.
        splitter ({'best', 'random'}): The strategy used to choose the split at each node.
        max_depth (int): The maximum depth of the tree.
        min_samples_split (float 0 < x < 1): Fraction of total sample representing minimum number of samples required to split an internal node.
        min_samples_leaf (float 0 < x < 1): Fraction of total sample representing minimum number of samples for each node.

    Returns:
        np.ndarray: An array with model predictions.
    """
    
    if min_samples_split is None:
        min_samples_split = 2

    if min_samples_leaf is None:
        min_samples_leaf = 1

    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = DecisionTreeClassifier(
        criterion = criterion,
        splitter = splitter,
        max_depth = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        random_state = 42
    )
    Model.fit(X = X, y = y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:, 1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred

def random_forest(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        n_estimators : int = 100,
        criterion : Literal['gini', 'entropy', 'log_loss'] = 'gini',
        max_depth : int = None,
        min_samples_split : float = None,
        min_samples_leaf : float = None
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses random forest algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        n_estimators (int): The number of trees in the forest.
        criterion ({'gini', 'entropy', 'log_loss'}): The function to measure the quality of a split.
        max_depth (int): The maximum depth of the tree.
        min_samples_split (float 0 < x < 1): Fraction of total sample representing minimum number of samples required to split an internal node.
        min_samples_leaf (float 0 < x < 1): Fraction of total sample representing minimum number of samples for each node.

    Returns:
        np.ndarray: An array with model predictions.
    """

    if min_samples_split is None:
        min_samples_split = 2

    if min_samples_leaf is None:
        min_samples_leaf = 1

    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = RandomForestClassifier(
        n_estimators = n_estimators,
        criterion = criterion,
        max_depth = max_depth,
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf,
        random_state = 42
    )
    Model.fit(X = X, y = y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:, 1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred

def light_gradient_boosting_machine(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        num_iterations : int = 100,
        learning_rate : float = 0.1,
        num_leaves : int = 31,
        max_depth : int = -1,
        bagging_freq : int = 0,
        bagging_fraction : float = 1,
        feature_fraction : float = 1,
        lambda_l1 : float = 0,
        lambda_l2 : float = 0,
        min_split_gain : float = 0
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses light gradient boosting machine algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        num_iterations (int): The number of boosting iterations.
        learning_rate (float > 0): Shrinkage rate.
        num_leaves (int 1 < x <= 131072): The max number of leaves in one tree.
        max_depth (int): Limit the max depth for tree model. <= 0 means no limit.
        bagging_freq (int >= 0): Frequency for bagging.
        bagging_fraction (float 0 < x <= 1): To randomly selecting data without resampling.
        feature_fraction (float 0 < x <= 1): To randomly selecting data without resampling.
        lambda_l1 (float >= 0): L1 regularization.
        lambda_l2 (float >= 0): L2 regularization.
        min_split_gain (float >= 0): The minimal gain to perform split.

    Returns:
        np.ndarray: An array with model predictions.
    """

    y = train_data[response_col].astype(int)
    X = train_data.drop(columns = [response_col])

    train_data = lgb.Dataset(X, label = y)
    param = {
        'objective' : 'binary', 
        'metric' : 'binary_logloss',
        'boosting_type' : 'gbdt',
        'num_iterations' : num_iterations,
        'learning_rate' : learning_rate,
        'num_leaves' : num_leaves,
        'max_depth' : max_depth,
        'bagging_freq' : bagging_freq,
        'bagging_fraction' : bagging_fraction,
        'feature_fraction' : feature_fraction,
        'lambda_l1' : lambda_l1,
        'lambda_l2' : lambda_l2,
        'min_split_gain': min_split_gain,
        'deterministic' : True,
        'verbosity' : -1
    }
    bst = lgb.train(param, train_data, 100)

    feature_names = X.columns
    y_prob = bst.predict(test_data[feature_names])
    if prob:
        return y_prob
    y_pred = np.array([1 if prob > 0.5 else 0 for prob in y_prob])

    return y_pred

def stochastic_gradient_descent(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        penalty : Literal['l2', 'l1', 'elasticnet', None] = 'l2',
        alpha : float = 0.0001,
        max_iter : int = 100,
        learning_rate : Literal['constant', 'optimal', 'invscaling', 'adaptive'] = 'optimal'
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses stochastic gradient descent algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        penalty ({'l2', 'l1', 'elasticnet', None}): The penalty (aka regularization term) to be used. 
        alpha (float > 0): Constant that multiplies the regularization term.
        max_iter (int): The maximum number of passes over the training data (aka epochs).
        learning_rate ({'constant', 'optimal', 'invscaling', 'adaptive'}): The learning rate.

    Returns:
        np.ndarray: An array with model predictions.
    """

    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = SGDClassifier(
        loss = "log_loss", 
        penalty = penalty,
        alpha = alpha,
        max_iter = max_iter,
        learning_rate = learning_rate,
        random_state = 42)
    Model.fit(X, y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:, 1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred

def extreme_gradient_boost(
        train_data : pd.DataFrame,
        response_col : str,
        test_data : pd.DataFrame,
        prob : bool = False,
        learning_rate : float = 0.3,
        min_split_loss : float = 0,
        max_depth : int = 6,
        max_delta_step : int = 0,
        subsample : float = 1,
        sampling_method : Literal['uniform', 'gradient_based'] = 'uniform',
        reg_lambda : float = 1,
        reg_alpha : float = 0,
        tree_method : Literal['auto', 'exact', 'approx', 'hist'] = 'auto'
    ) -> np.ndarray[np.int64 | np.float64]:
    """
    Uses extreme gradient boost algorithm to perform classification.

    Parameters:
        train_data (pd.DataFrame): The input DataFrame containing the train data.
        response_col (str): The name of the target column (class label).
        test_data (pd.DataFrame): The input DataFrame containing the test data.
        prob (bool): To return a vector of probabilities instead of class predictions.
        learning_rate (float 0 <= x <= 1): Step size shrinkage used in update to prevent overfitting.
        min_split_loss (float >= 0): Minimum loss reduction required to make a further partition on a leaf node of the tree.
        max_depth (int): Maximum depth of a tree. Beware that XGBoost aggressively consumes memory when training a deep tree. "exact" tree method requires non-zero value.
        max_delta_step (int >= 0): Maximum delta step we allow each leaf output to be. Setting it to value of 1-10 might help control the update.
        subsample (int 0 < x <= 1): Subsample ratio of the training instances.
        sampling_method ({'uniform', 'gradient_based'}): The method to use to sample the training instances.
        reg_lambda (float >= 0): L2 regularization term on weights. 
        reg_alpha (float >= 0): L1 regularization term on weights. 
        tree_method ({'auto', 'exact', 'approx', 'hist'}): The tree construction algorithm used in XGBoost.

    Returns:
        np.ndarray: An array with model predictions.
    """

    y = train_data[response_col]
    X = train_data.drop(columns = [response_col])

    Model = xgb.XGBClassifier(
        eta = learning_rate,
        gamma = min_split_loss,
        max_depth = max_depth,
        max_delta_step = max_delta_step,
        subsample = subsample,
        sampling_method = sampling_method,
        reg_lambda = reg_lambda,
        reg_alpha = reg_alpha,
        tree_method = tree_method,
        random_state = 42
    )
    Model.fit(X, y)

    feature_names = X.columns
    if prob:
        y_pred = Model.predict_proba(test_data[feature_names])[:,1]
    else:
        y_pred = Model.predict(test_data[feature_names])

    return y_pred


In [12]:
## Hyperparameter Tuning functions

def cross_validate_model(
        model : Callable,
        params : dict,
        data : pd.DataFrame = data_processing(),
        response_col : str = 'target',
        n_folds : int = 5
    ) -> dict:
    """
    Performs stratified K-fold cross-validation on a specific model,
    ensuring the ratio of positive and negative cases is balanced across all folds.
    
    Parameters:
        data (pd.DataFrame): The full dataset to split into folds.
        response_col (str): The name of the response column.
        params (dict): The parameters for the model.
        n_folds (int): The number of folds for cross-validation.
        
    Returns:
        dict: A dictionary containing average cross-validated metrics.
    """
    
    skf = StratifiedKFold(n_splits = n_folds, shuffle = True, random_state = 42)
    
    recall_scores = []
    
    X = data.drop(columns = [response_col])
    y = data[response_col]

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        train_data = pd.concat([X_train, y_train], axis = 1)

        test_data = X_test
        
        y_pred = model(
            train_data = train_data, 
            response_col = response_col, 
            test_data = test_data,
            prob = False,
            **params
        )
        
        recall_scores.append(recall_score(y_test, y_pred, zero_division = 0))
    
    recall = np.mean(recall_scores)
    
    return recall

def get_optimal_parameters(
        model : Callable,
        int_params : dict[str : list] = {},
        float_params : dict[str : list] = {},
        categorical_params : dict[str : list] = {},
        trials : int = 20
        ):

    def helper(
            trial : optuna.trial.Trial,
        ) -> dict:

        params = {}

        for argument, values in int_params.items():
            params[argument] = trial.suggest_int(argument, values[0], values[1])
        
        for argument, values in float_params.items():
            params[argument] = trial.suggest_float(argument, values[0], values[1])

        for argument, values in categorical_params.items():
            params[argument] = trial.suggest_categorical(argument, values)

        recall = cross_validate_model(
            model = model,
            params = params
        )

        return(recall)

    study = optuna.create_study(direction = 'maximize')
    study.optimize(func = helper, n_trials = trials)
    return(study.best_params)


In [13]:
## Define main workflow

def main():
    df = data_processing()

In [14]:
## Run to execute main workflow

main()


In [15]:
## FOR TESTING 

get_optimal_parameters(
    light_gradient_boosting_machine,
    int_params = {
        'num_iterations' : [50, 200],
        'num_leaves' : [1, 50],
        'max_depth' : [1, 50],
        'bagging_freq' : [0, 10]
    },
    float_params = {
        'learning_rate' : [0.001, 10],
        'bagging_fraction' : [0.001, 1],
        'feature_fraction' : [0.001, 1],
        'lambda_l1' : [0, 5],
        'lambda_l2' : [0, 5],
        'min_split_gain' : [0, 5]
    },
    categorical_params = {}
)

[I 2024-10-25 11:35:34,655] A new study created in memory with name: no-name-8fa00331-86a9-4de7-810f-c53db60a513d
[I 2024-10-25 11:35:35,147] Trial 0 finished with value: 0.8433085501858736 and parameters: {'num_iterations': 125, 'num_leaves': 47, 'max_depth': 15, 'bagging_freq': 4, 'learning_rate': 0.44941056593320144, 'bagging_fraction': 0.8366631481382854, 'feature_fraction': 0.2824151813841448, 'lambda_l1': 3.093668648684237, 'lambda_l2': 1.6053308362939156, 'min_split_gain': 0.3941113237774402}. Best is trial 0 with value: 0.8433085501858736.
[I 2024-10-25 11:35:35,579] Trial 1 finished with value: 0.5137546468401487 and parameters: {'num_iterations': 193, 'num_leaves': 44, 'max_depth': 24, 'bagging_freq': 3, 'learning_rate': 7.610558993795522, 'bagging_fraction': 0.5137357739810291, 'feature_fraction': 0.8213061493716085, 'lambda_l1': 1.0604475047366595, 'lambda_l2': 3.8820102600026725, 'min_split_gain': 3.1743073598836484}. Best is trial 0 with value: 0.8433085501858736.
[I 2024

{'num_iterations': 125,
 'num_leaves': 47,
 'max_depth': 15,
 'bagging_freq': 4,
 'learning_rate': 0.44941056593320144,
 'bagging_fraction': 0.8366631481382854,
 'feature_fraction': 0.2824151813841448,
 'lambda_l1': 3.093668648684237,
 'lambda_l2': 1.6053308362939156,
 'min_split_gain': 0.3941113237774402}